# Object Detection Changed For Video


# Imports

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')

# add opencv for video cutting
import cv2



## Env setup

In [532]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [533]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [534]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'

#MODEL_NAME = 'ssd_inception_v2_coco_11_06_2017'

#MODEL_NAME = 'rfcn_resnet101_coco_11_06_2017'

#MODEL_NAME = 'faster_rcnn_resnet101_coco_11_06_2017'

#MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous_coco_11_06_2017'

MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

## Download Model

In [535]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [536]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [537]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## Helper code

In [538]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [539]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [540]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]

       
        
  return output_dict

In [541]:
def process_image(image):
    plt.imsave('in.png',image)
    im = Image.open("in.png")
    im.save("in.jpg")
    image = Image.open("in.jpg")
    image_np = load_image_into_numpy_array(image)
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
    plt.imsave('res.png',image_np)
    imageres=cv2.imread('res.png')
    return imageres



In [542]:
# Import everything needed to edit/save/watch video clips
import imageio
imageio.plugins.ffmpeg.download()

from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [543]:
white_output = 'video1_out.mp4'# Change HERE for video input and time 
clip1 = VideoFileClip("video1.mp4").subclip(0,60)# subclip(start seconds, end seconds)

In [544]:
clip_detected = clip1.fl_image(process_image) 
%time clip_detected.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video video1_out.mp4
[MoviePy] Writing video video1_out.mp4





  0%|          | 0/1501 [00:00<?, ?it/s]


  0%|          | 1/1501 [00:04<2:03:50,  4.95s/it]


  0%|          | 2/1501 [00:10<2:05:04,  5.01s/it]


  0%|          | 3/1501 [00:15<2:04:40,  4.99s/it]


  0%|          | 4/1501 [00:19<2:04:04,  4.97s/it]


  0%|          | 5/1501 [00:24<2:03:36,  4.96s/it]


  0%|          | 6/1501 [00:30<2:05:00,  5.02s/it]


  0%|          | 7/1501 [00:35<2:10:39,  5.25s/it]


  1%|          | 8/1501 [00:41<2:14:13,  5.39s/it]


  1%|          | 9/1501 [00:47<2:17:58,  5.55s/it]


  1%|          | 10/1501 [00:52<2:16:42,  5.50s/it]


  1%|          | 11/1501 [00:58<2:19:14,  5.61s/it]


  1%|          | 12/1501 [01:05<2:25:31,  5.86s/it]


  1%|          | 13/1501 [01:11<2:27:44,  5.96s/it]


  1%|          | 14/1501 [01:17<2:29:11,  6.02s/it]


  1%|          | 15/1501 [01:25<2:44:32,  6.64s/it]


  1%|          | 16/1501 [01:31<2:42:06,  6.55s/it]


  1%|          | 17/1501 [01:37<2:36:20,  6.32s/it]


  1%|          | 18/1501 [01:44<2:37:51,  6.3

  9%|▊         | 129/1501 [12:50<2:28:37,  6.50s/it]


  9%|▊         | 130/1501 [12:56<2:27:29,  6.45s/it]


  9%|▊         | 131/1501 [13:03<2:26:24,  6.41s/it]


  9%|▉         | 132/1501 [13:08<2:22:12,  6.23s/it]


  9%|▉         | 133/1501 [13:14<2:21:09,  6.19s/it]


  9%|▉         | 134/1501 [13:21<2:26:47,  6.44s/it]


  9%|▉         | 135/1501 [13:27<2:23:04,  6.28s/it]


  9%|▉         | 136/1501 [13:33<2:20:10,  6.16s/it]


  9%|▉         | 137/1501 [13:40<2:26:37,  6.45s/it]


  9%|▉         | 138/1501 [13:46<2:24:05,  6.34s/it]


  9%|▉         | 139/1501 [13:52<2:21:22,  6.23s/it]


  9%|▉         | 140/1501 [14:00<2:29:06,  6.57s/it]


  9%|▉         | 141/1501 [14:07<2:32:43,  6.74s/it]


  9%|▉         | 142/1501 [14:13<2:29:16,  6.59s/it]


 10%|▉         | 143/1501 [14:19<2:27:23,  6.51s/it]


 10%|▉         | 144/1501 [14:25<2:22:00,  6.28s/it]


 10%|▉         | 145/1501 [14:32<2:22:43,  6.32s/it]


 10%|▉         | 146/1501 [14:39<2:27:47,  6.54s/it]


 10%|▉    

 17%|█▋        | 257/1501 [28:53<2:29:49,  7.23s/it]


 17%|█▋        | 258/1501 [29:00<2:28:58,  7.19s/it]


 17%|█▋        | 259/1501 [29:06<2:23:01,  6.91s/it]


 17%|█▋        | 260/1501 [29:14<2:26:01,  7.06s/it]


 17%|█▋        | 261/1501 [29:23<2:38:25,  7.67s/it]


 17%|█▋        | 262/1501 [29:30<2:34:57,  7.50s/it]


 18%|█▊        | 263/1501 [29:39<2:42:14,  7.86s/it]


 18%|█▊        | 264/1501 [29:47<2:43:31,  7.93s/it]


 18%|█▊        | 265/1501 [29:53<2:33:24,  7.45s/it]


 18%|█▊        | 266/1501 [30:00<2:28:53,  7.23s/it]


 18%|█▊        | 267/1501 [30:08<2:32:25,  7.41s/it]


 18%|█▊        | 268/1501 [30:14<2:25:12,  7.07s/it]


 18%|█▊        | 269/1501 [30:20<2:20:26,  6.84s/it]


 18%|█▊        | 270/1501 [30:26<2:17:04,  6.68s/it]


 18%|█▊        | 271/1501 [30:35<2:26:01,  7.12s/it]


 18%|█▊        | 272/1501 [30:42<2:25:34,  7.11s/it]


 18%|█▊        | 273/1501 [30:51<2:38:24,  7.74s/it]


 18%|█▊        | 274/1501 [30:58<2:34:21,  7.55s/it]


 18%|█▊   

 26%|██▌       | 385/1501 [43:49<2:07:44,  6.87s/it]


 26%|██▌       | 386/1501 [43:56<2:05:34,  6.76s/it]


 26%|██▌       | 387/1501 [44:04<2:13:02,  7.17s/it]


 26%|██▌       | 388/1501 [44:12<2:16:41,  7.37s/it]


 26%|██▌       | 389/1501 [44:18<2:10:07,  7.02s/it]


 26%|██▌       | 390/1501 [44:24<2:05:34,  6.78s/it]


 26%|██▌       | 391/1501 [44:31<2:02:16,  6.61s/it]


 26%|██▌       | 392/1501 [44:38<2:09:33,  7.01s/it]


 26%|██▌       | 393/1501 [44:45<2:06:28,  6.85s/it]


 26%|██▌       | 394/1501 [44:51<2:02:57,  6.66s/it]


 26%|██▋       | 395/1501 [44:59<2:10:42,  7.09s/it]


 26%|██▋       | 396/1501 [45:06<2:06:14,  6.85s/it]


 26%|██▋       | 397/1501 [45:12<2:04:29,  6.77s/it]


 27%|██▋       | 398/1501 [45:20<2:10:29,  7.10s/it]


 27%|██▋       | 399/1501 [45:26<2:05:30,  6.83s/it]


 27%|██▋       | 400/1501 [45:33<2:05:44,  6.85s/it]


 27%|██▋       | 401/1501 [45:41<2:10:02,  7.09s/it]


 27%|██▋       | 402/1501 [45:47<2:04:54,  6.82s/it]


 27%|██▋  

...

 67%|██████▋   | 1009/1501 [1:57:58<1:01:55,  7.55s/it]


 67%|██████▋   | 1010/1501 [1:58:05<1:00:39,  7.41s/it]


 67%|██████▋   | 1011/1501 [1:58:12<59:36,  7.30s/it]  


 67%|██████▋   | 1012/1501 [1:58:19<56:53,  6.98s/it]


 67%|██████▋   | 1013/1501 [1:58:25<55:04,  6.77s/it]


 68%|██████▊   | 1014/1501 [1:58:31<53:45,  6.62s/it]


 68%|██████▊   | 1015/1501 [1:58:39<57:11,  7.06s/it]


 68%|██████▊   | 1016/1501 [1:58:46<56:49,  7.03s/it]


 68%|██████▊   | 1017/1501 [1:58:55<1:01:52,  7.67s/it]


 68%|██████▊   | 1018/1501 [1:59:02<1:00:16,  7.49s/it]


 68%|██████▊   | 1019/1501 [1:59:09<59:08,  7.36s/it]  


 68%|██████▊   | 1020/1501 [1:59:16<56:10,  7.01s/it]


 68%|██████▊   | 1021/1501 [1:59:23<57:11,  7.15s/it]


 68%|██████▊   | 1022/1501 [1:59:31<59:20,  7.43s/it]


 68%|██████▊   | 1023/1501 [1:59:38<58:28,  7.34s/it]


 68%|██████▊   | 1024/1501 [1:59:45<57:37,  7.25s/it]


 68%|██████▊   | 1025/1501 [1:59:52<56:50,  7.17s/it]


 68%|██████▊   | 1026/1501 [1:59:59<

 76%|███████▌  | 1134/1501 [2:13:24<43:39,  7.14s/it]


 76%|███████▌  | 1135/1501 [2:13:32<45:05,  7.39s/it]


 76%|███████▌  | 1136/1501 [2:13:40<46:06,  7.58s/it]


 76%|███████▌  | 1137/1501 [2:13:48<46:44,  7.70s/it]


 76%|███████▌  | 1138/1501 [2:13:56<47:05,  7.78s/it]


 76%|███████▌  | 1139/1501 [2:14:03<45:27,  7.53s/it]


 76%|███████▌  | 1140/1501 [2:14:12<48:00,  7.98s/it]


 76%|███████▌  | 1141/1501 [2:14:19<46:00,  7.67s/it]


 76%|███████▌  | 1142/1501 [2:14:26<44:32,  7.45s/it]


 76%|███████▌  | 1143/1501 [2:14:33<43:26,  7.28s/it]


 76%|███████▌  | 1144/1501 [2:14:40<42:42,  7.18s/it]


 76%|███████▋  | 1145/1501 [2:14:49<45:51,  7.73s/it]


 76%|███████▋  | 1146/1501 [2:14:56<44:12,  7.47s/it]


 76%|███████▋  | 1147/1501 [2:15:02<43:04,  7.30s/it]


 76%|███████▋  | 1148/1501 [2:15:09<42:15,  7.18s/it]


 77%|███████▋  | 1149/1501 [2:15:16<41:58,  7.15s/it]


 77%|███████▋  | 1150/1501 [2:15:25<44:52,  7.67s/it]


 77%|███████▋  | 1151/1501 [2:15:32<43:16,  7.42

 84%|████████▍ | 1260/1501 [2:28:57<31:20,  7.80s/it]


 84%|████████▍ | 1261/1501 [2:29:05<31:33,  7.89s/it]


 84%|████████▍ | 1262/1501 [2:29:13<31:31,  7.91s/it]


 84%|████████▍ | 1263/1501 [2:29:19<29:24,  7.41s/it]


 84%|████████▍ | 1264/1501 [2:29:26<28:20,  7.17s/it]


 84%|████████▍ | 1265/1501 [2:29:34<28:55,  7.35s/it]


 84%|████████▍ | 1266/1501 [2:29:41<28:22,  7.24s/it]


 84%|████████▍ | 1267/1501 [2:29:50<30:28,  7.81s/it]


 84%|████████▍ | 1268/1501 [2:29:57<29:17,  7.54s/it]


 85%|████████▍ | 1269/1501 [2:30:04<28:33,  7.38s/it]


 85%|████████▍ | 1270/1501 [2:30:11<28:01,  7.28s/it]


 85%|████████▍ | 1271/1501 [2:30:17<26:34,  6.93s/it]


 85%|████████▍ | 1272/1501 [2:30:24<26:54,  7.05s/it]


 85%|████████▍ | 1273/1501 [2:30:35<30:36,  8.05s/it]


 85%|████████▍ | 1274/1501 [2:30:42<30:05,  7.95s/it]


 85%|████████▍ | 1275/1501 [2:30:50<29:21,  7.79s/it]


 85%|████████▌ | 1276/1501 [2:30:58<30:22,  8.10s/it]


 85%|████████▌ | 1277/1501 [2:31:05<28:50,  7.72

 92%|█████████▏| 1386/1501 [2:44:30<15:31,  8.10s/it]


 92%|█████████▏| 1387/1501 [2:44:39<15:49,  8.33s/it]


 92%|█████████▏| 1388/1501 [2:44:46<14:47,  7.85s/it]


 93%|█████████▎| 1389/1501 [2:44:52<13:40,  7.32s/it]


 93%|█████████▎| 1390/1501 [2:45:00<13:44,  7.43s/it]


 93%|█████████▎| 1391/1501 [2:45:08<13:54,  7.59s/it]


 93%|█████████▎| 1392/1501 [2:45:15<13:22,  7.37s/it]


 93%|█████████▎| 1393/1501 [2:45:23<14:01,  7.79s/it]


 93%|█████████▎| 1394/1501 [2:45:31<13:44,  7.71s/it]


 93%|█████████▎| 1395/1501 [2:45:38<13:20,  7.56s/it]


 93%|█████████▎| 1396/1501 [2:45:47<13:55,  7.96s/it]


 93%|█████████▎| 1397/1501 [2:45:54<13:12,  7.62s/it]


 93%|█████████▎| 1398/1501 [2:46:01<12:42,  7.41s/it]


 93%|█████████▎| 1399/1501 [2:46:08<12:21,  7.27s/it]


 93%|█████████▎| 1400/1501 [2:46:15<12:03,  7.16s/it]


 93%|█████████▎| 1401/1501 [2:46:22<11:50,  7.10s/it]


 93%|█████████▎| 1402/1501 [2:46:31<12:41,  7.69s/it]


 93%|█████████▎| 1403/1501 [2:46:37<12:09,  7.45

[MoviePy] Done.
[MoviePy] >>>> Video ready: video1_out.mp4 

CPU times: user 3h 5min 56s, sys: 53.5 s, total: 3h 6min 50s
Wall time: 2h 58min 50s
